In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import googlemaps
import gmplot

from dotenv import load_dotenv

from src.som import som

load_dotenv()
API_KEY = os.getenv('API_KEY')

In [12]:
gclient = googlemaps.Client(API_KEY)

In [3]:
with open('enderecos.txt', 'r') as f:
    enderecos = f.readlines()
    end_list = [texto.replace('\n', '') for texto in enderecos]

In [4]:
coordinates = []

lats = []
lgns = []

for end in end_list:
    geocode_result = gclient.geocode(end)
    coordinates.append(geocode_result[0]['geometry']['location'])
    
    lats.append(geocode_result[0]['geometry']['location']['lat'])
    lgns.append(geocode_result[0]['geometry']['location']['lng'])

In [5]:
apikey = API_KEY

gmap = gmplot.GoogleMapPlotter(
    np.mean(lats),
    np.mean(lgns),
    13,
    apikey=apikey
)

gmap.scatter(lats, lgns, color='red')
gmap.draw('maps/locations.html')

In [6]:
df_coords = pd.DataFrame(coordinates)
df_coords.columns = ['x', 'y']
df_coords['codigo'] = df_coords.index

In [7]:
route = som(df_coords, 100000, 'teste')

Network of 64 neurons created. Starting the iterations:
Radius has completely decayed, finishing execution at 13860 iterations


In [8]:
route

Int64Index([5, 7, 2, 0, 3, 6, 1, 4], dtype='int64')

In [9]:
start = 0
lst_route = list(route)

idx = lst_route.index(start)

last = lst_route[idx:]
first = lst_route[:idx]

final = last + first

In [37]:
location_list = []
for i, (_, row) in enumerate(df_coords.iloc[final].iterrows()):
    if i == 0 :
        start = (row['x'], row['y'])
    elif i == len(df_coords) - 1:
        end = (row['x'], row['y'])
    else:
        location_list.append((row['x'], row['y']))

In [11]:
gmap = gmplot.GoogleMapPlotter(
    np.mean(lats),
    np.mean(lgns),
    13,
    apikey=API_KEY
)

gmap.directions(
    start,
    end,
    waypoints=location_list
)

gmap.draw('maps/map.html')

In [38]:
distance = 0
time = 0

location_list.insert(0, start)
location_list.insert(len(location_list) + 1, end)

for i in range(len(location_list) - 1):
    response = gclient.distance_matrix(location_list[i], location_list[i+1])
    
    distance += response['rows'][0]['elements'][0]['distance']['value']
    time += response['rows'][0]['elements'][0]['duration']['value']
    

In [39]:
distance / 1000

28277

In [40]:
time / 60

3948